In [1]:
import pandas as pd
import numpy as np
import spacy as sp
from spacy.lang.en.stop_words import STOP_WORDS
from spacy.language import Language
from keras.preprocessing.text import Tokenizer



In [2]:
data = pd.read_json(r"C:\Users\leofr\VsCode\CS663\CS663-Final-Project\CS663-Final-Project\CS663_Project_Data.json", orient='index')
data = data

In [3]:
df = data.copy()
df

,invocation,cmd
1,"Copy loadable kernel module ""mymodule.ko"" to t...",sudo cp mymodule.ko /lib/modules/$(uname -r)/k...
2,"Display all lines containing ""IP_MROUTE"" in th...",cat /boot/config-`uname -r` | grep IP_MROUTE
3,Display current running kernel's compile-time ...,cat /boot/config-`uname -r`
4,"Find all loadable modules for current kernel, ...",find /lib/modules/`uname -r` -regex .*perf.*
5,"Look for any instance of ""HIGHMEM"" in the curr...",grep “HIGHMEM” /boot/config-`uname -r`
...,...,...
10343,using exec in find command to dispaly the sear...,find . ... -exec cat {} \; -exec echo \;
10344,verbosely create intermediate directoriy tmp a...,mkdir -pv /tmp/boostinst
10345,view the manual page of find,man find
10346,"wait 2 seconds and then print ""hello""","echo ""hello `sleep 2 &`"""


In [4]:
nlp = sp.load('en_core_web_sm')

@Language.component('clean_inv')
def clean_inv(doc):
    # Lowercase
    doc = [token.lower_ for token in doc]
    # # Punctuation + special chars
    # tokens = [token for token in tokens if not token.is_punct and not token.is_space and not token.is_digit and not token.like_num and not token.is_currency and not token.is_stop]
    # # Stop words
    # tokens = [token for token in tokens if not token in STOP_WORDS]
    return doc

#nlp.add_pipe('clean_inv', name='clean_inv', last=True)

def tokenize_inv(inv):
    doc = nlp(inv)
    tokens = [token.text for token in doc]
    return tokens


In [5]:

df['inv_tokens'] = df['invocation'].apply(tokenize_inv)
df

,invocation,cmd,inv_tokens
1,"Copy loadable kernel module ""mymodule.ko"" to t...",sudo cp mymodule.ko /lib/modules/$(uname -r)/k...,"[Copy, loadable, kernel, module, "", mymodule.k..."
2,"Display all lines containing ""IP_MROUTE"" in th...",cat /boot/config-`uname -r` | grep IP_MROUTE,"[Display, all, lines, containing, "", IP_MROUTE..."
3,Display current running kernel's compile-time ...,cat /boot/config-`uname -r`,"[Display, current, running, kernel, 's, compil..."
4,"Find all loadable modules for current kernel, ...",find /lib/modules/`uname -r` -regex .*perf.*,"[Find, all, loadable, modules, for, current, k..."
5,"Look for any instance of ""HIGHMEM"" in the curr...",grep “HIGHMEM” /boot/config-`uname -r`,"[Look, for, any, instance, of, "", HIGHMEM, "", ..."
...,...,...,...
10343,using exec in find command to dispaly the sear...,find . ... -exec cat {} \; -exec echo \;,"[using, exec, in, find, command, to, dispaly, ..."
10344,verbosely create intermediate directoriy tmp a...,mkdir -pv /tmp/boostinst,"[verbosely, create, intermediate, directoriy, ..."
10345,view the manual page of find,man find,"[view, the, manual, page, of, find]"
10346,"wait 2 seconds and then print ""hello""","echo ""hello `sleep 2 &`""","[wait, 2, seconds, and, then, print, "", hello, ""]"


In [6]:
invocations = df['inv_tokens'].values
commands = df['cmd'].values

# Define the maximum number of words to keep based on word frequency
max_words = 10000
# Create a tokenizer and fit it on the tokenized text
tokenizer = Tokenizer(num_words=max_words)
tokenizer.fit_on_texts(invocations)

sequences = tokenizer.texts_to_sequences(invocations)
target_sequences = tokenizer.texts_to_sequences(commands)

In [7]:
sequences

[[86, 2253, 463, 1508, 2, 3264, 2, 13, 1, 1805, 4, 751, 7, 3265, 9, 463, 16],
 [31, 5, 45, 116, 2, 3266, 2, 4, 1, 9, 463, 136, 1316, 30, 164, 653, 14, 16],
 [31, 9, 431, 463, 136, 1316, 30, 164, 653, 14, 16],
 [11, 5, 2253, 751, 21, 9, 463, 19, 59, 38, 1088, 2, 3267, 2],
 [370,
  21,
  123,
  1089,
  12,
  2,
  3268,
  2,
  4,
  1,
  9,
  463,
  136,
  1316,
  30,
  164,
  653,
  14,
  16],
 [24, 21, 85, 2, 2254, 2, 4, 1, 3269, 12, 1, 209, 15, 614, 3270],
 [31, 5, 45, 116, 3271, 4, 1, 9, 463, 136, 1316, 30, 164, 653, 14, 16],
 [31, 5, 45, 116, 3272, 4, 1, 9, 463, 136, 1316, 30, 164, 653, 14, 16],
 [370, 21, 123, 1089, 12, 2, 3273, 2, 4, 1, 3274, 14, 117, 9, 463, 719],
 [22, 239, 30, 3275, 65, 133, 15, 700],
 [3276, 1, 149, 63, 192, 65, 1, 76, 2255, 85],
 [297, 2, 51, 12, 98, 432, 2, 65, 283, 12, 55, 26, 418, 3, 17, 9, 7],
 [297, 2, 51, 12, 98, 432, 2, 65, 283, 12, 55, 26, 418, 3, 17, 9, 7],
 [297, 2, 169, 2, 13, 1, 78, 12, 5, 18, 4, 1, 83, 7, 23],
 [297,
  2,
  240,
  16,
  2,
  13,
  